Add the logistic regression score trained on the chosen combination of features.

In [1]:
import json
from sklearn.linear_model import LogisticRegression

In [3]:
path_train = 'scores/train.jl'

In [4]:
def read_rows(path):
    rows = []
    for line in open(path):
        rows.append(json.loads(line))
    return rows

def write_json_format(path_out, rows):
    f_out = open(path_out, 'w')
    for row in rows:
        f_out.write(json.dumps(row, ensure_ascii=False)+'\n')

rows_train = read_rows(path_train)

In [6]:
X_train = [[
    row['score_bm25'], 
    row['score_bm25_not_lemmatized'], 
    row['score_bm25_bigrams'], 
    row['score_miniLM'],
] for row in rows_train]
y_train = [row['label'] for row in rows_train]

In [7]:
model = LogisticRegression().fit(X_train, y_train)

In [41]:
path_test = 'scores/test-allegro.jl'
rows_test = read_rows(path_test)

In [42]:
X_test = [[
    row['score_bm25'], 
    row['score_bm25_not_lemmatized'], 
    row['score_bm25_bigrams'], 
    row['score_miniLM'],
] for row in rows_test]

In [43]:
probs = model.predict_proba(X_test)

In [44]:
for row, prob in zip(rows_test, probs):
    row['log_reg_score'] = prob[1]

In [45]:
len(rows_test), len(probs)

(39988, 39988)

In [46]:
rows_test[:10]

[{'question_id': 0,
  'question_text': 'Otrzymałem rekompensatę z POK, a później zwrot od Sprzedającego. Co mam zrobić?',
  'passage_id': '511',
  'passage_text': 'Jeśli wypłaciliśmy Ci rekompensatę w ramach Programu Ochrony Kupujących a Ty otrzymasz zwrot pieniędzy od Sprzedającego, masz obowiązek zwrócić nam rekompensatę.\nZgodnie z punktem 6 Część IV Załącznika nr 9 do Regulaminu Allegro, na zwrot rekompensaty masz 7 dni od naprawienia szkody przez Sprzedającego.',
  'score_bm25': 27.625212,
  'score_bm25_not_lemmatized': 19.79850458073086,
  'score_bm25_bigrams': 5.10528997024767,
  'passage_translated': "If we have paid you compensation under the Buyer Protection Programme and you receive a refund from the Seller, you are obliged to pay us compensation. According to Section 6 of Part IV of Annex No 9 to the Allegro Regulations, you have 7 days from the Seller's compensation.",
  'question_translated': 'I received compensation from the POK and then a refund from the Seller. What sh

In [47]:
write_json_format(path_test, rows_test)